In [ ]:
!pip install simpletransformers

import numpy as np
import pandas as pd

import torch
import torch.nn as nn

from simpletransformers.ner import NERModel
from simpletransformers.classification import ClassificationModel


# Load Dataset

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
cols = ["text","label"]
df = pd.read_csv(
    "/content/drive/MyDrive/Bachelor/df_NO.csv",
    header=0,
    engine="python",
    encoding="UTF-8"
)
df = df[["Transcription", "Backchannel", ".folds", "index"]]
df.columns = ["text", "label", "fold", "num"]

Mounted at /content/drive


In [ ]:
device = torch.device("cuda")

In [ ]:
prs = [1,2,3,4,5,6,7,8,9,10]

In [ ]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [ ]:
for i in prs:
  testdata = df[df["fold"] == i]
  traindata = df[df["fold"]!=i]

  n_labels = len(df['label'].unique())

  # initialize the model
  sent_model = ClassificationModel('bert', "ViktorAlm/electra-base-norwegian-uncased-discriminator",
                                 num_labels=n_labels, use_cuda=True,
                                 args={'reprocess_input_data': True, 'overwrite_output_dir': True,
                                       "num_train_epochs": 10, 'max_seq_length': 32})
  sent_model.train_model(traindata)

  result, model_outputs, wrong_predictions = sent_model.eval_model(testdata, acc=sklearn.metrics.accuracy_score)

  testdata.index = [i for i in range(len(testdata.text))]

  predictions, raw_outputs = sent_model.predict(testdata.text)
  print(pd.crosstab(testdata.label, predictions))
  df_res = pd.DataFrame(list(zip(testdata.num, predictions)), 
               columns =['index', 'bert_preds'])
  filename = [str(i), "fold", "_NO_df.csv"]
  df_res.to_csv("_".join(filename))
  